In [1]:
import pandas as pd
from sklearn.model_selection import GroupKFold, KFold
import itertools
import functools
import numpy as np

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.svm import SVC
import xgboost as xgb
#from thundersvm import SVC # need to build library! xcode-select --install then 
# https://thundersvm.readthedocs.io/en/latest/get-started.html

Planning:

-ML
    -Fix c++ compiler path for Thunder SVM...
    -Test vanilla SVM and XGboost

-DC
    -What are inputs? nl_03 plus: Smiles/RDkit
    -Dataset formating: Passing dataset object?
    -dataset.X, dataset.y, dataset.w

-Profile scripts for slow steps per Lachlan's instructions

-Other
    -Simple probability of loss from observed data...
    -Model from standards only
    -Combine model?  Has water then apply ML model



In [2]:
join_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/all_public_output_02.pickle')

In [6]:
# Fixes issue that bits column for FP4 fingerprints is list of np.64 instead of 
# np.array of np.bools
# Moveed to end of nl_2_join.py
join_df.bits = join_df.bits.apply(lambda x: np.asarray(x, dtype=np.bool))

# Check for bad bits (FP4 fingerprints)
# 6,115,379 / 6,121,864 are good!
def is_1024(x):
    if type(x) is np.ndarray and len(x) == 1024:
        return np.bool_(1)
    else:
        return np.nan

join_df['good_bits'] = join_df.bits.apply(lambda x: is_1024(x))
join_df = join_df.dropna(subset=['good_bits'])

In [7]:
join_df.shape

(6115379, 31)

In [ ]:
join_df.to_pickle('/Users/dis/PycharmProjects/neutral_loss/all_public_output_02b.pickle')
join_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/all_public_output_02b.pickle')

In [429]:
# Filtering:
target = ['H2O'] # ['H2O']
polarity = ['positive'] # ['positive', 'negative'] 
fdrs = [0.05] # [0.2, 0.1, 0.05]
colocalizations = [0.75] # [0, 0.5, 0.75]

# Outputs (y), and weights (w):
global_ys = ['n_loss_only_H2O', 'n_loss_wparent_H2O']
global_y = [global_ys[1]]
global_w = ['weight']

# Class of model:
direct_model_on = False
ml_model_on = True
deepchem_model_on = False

# Split.  If true, any formula only appears in train/test/validate.
single_fold_group = True

ml_df = direct_df = pd.DataFrame(columns=['target', 'polarity', 'fdr', 'coloc',
                                       'mclass', 'model', 'X', 'y', 'w'])
dc_df = ml_df

# Specific inputs (X) for each model:
if direct_model_on is True:
    model = ['']
    Xs = ['trues', 'falses', 'rando', 'H2O_Present', 'n_loss_wparent_H2O']
    X =  [Xs[3]
    y = global_y
    w = global_w
    direct_models = list(itertools.product(*[target, polarity, fdrs, colocalizations,
                                             ['direct'], model, X, y, w]))
    direct_df = pd.DataFrame(direct_models, columns=['target', 'polarity', 'fdr', 'coloc',
                                       'mclass', 'model', 'X', 'y', 'w'])

if ml_model_on is True:
    models = ['random_forest', 'XGBoost', 'ThunderSVM']
    model = [models[1]]
    Xs = ['bits', 'mord_norm', 'fp_feats'] 
    X = [Xs[1]]
    y = global_y
    w = global_w
    ml_models = list(itertools.product(*[target, polarity, fdrs, colocalizations,
                                         ['ml'], model, X, y, w]))
    ml_df = pd.DataFrame(ml_models, columns=['target', 'polarity', 'fdr', 'coloc',
                                       'mclass', 'model', 'X', 'y', 'w']) 

if deepchem_model_on is True:
    models = [('GraphConvModel', 'GraphConv'), 
              ('WeaveModel', 'Weave'), 
              ('WeaveFeaturizer','Weave') ] 
    model = [models[0]]    
    Xs = ['Molecule', 'Smiles']
    X = [Xs[0]]          
    y = global_y
    w = global_w
    dc_models = list(itertools.product(*[target, polarity, fdrs, colocalizations,
                                         ['dc'], model, X, y, w]))
    dc_df = pd.DataFrame(dc_models, columns=['target', 'polarity', 'fdr', 'coloc',
                                       'mclass', 'model', 'X', 'y', 'w'])
    
filter_param_df = pd.concat([direct_df, ml_df, dc_df]).reset_index(drop=True)
filter_param_df

,target,polarity,fdr,coloc,mclass,model,X,y,w
0,H2O,positive,0.05,0.75,ml,XGBoost,mord_norm,n_loss_wparent_H2O,weight


In [199]:
def sanitize(df):
    # Drops rows with nans and inconsistent types
    df = df.dropna(axis=0)
    return df

In [228]:
def split(df, single_fold):
    # True: will segregate formulas to one group or another.  Prevents memorization and 
    # enhances ability to predict new molecues.
    # False: is memorization a bad thing over 2900+ datasets?  Extinction plot for novel ID's
    # 50:25:25 / Train:Test:Val. Rationale: Chemical space is large.
    # Working!

    X = df
    y = df.y
    groups = df.formula

    if single_fold is True:
        splitter = GroupKFold(n_splits=2)
    else:
        splitter = KFold(n_splits=2)

    group_kfold = GroupKFold(n_splits=2)
    for train_ix, tv_ix in splitter.split(X, y, groups):
        X_train, X_tv = X.iloc[train_ix, :].copy(deep=True), X.iloc[tv_ix, :].copy(deep=True)

    X = X_tv
    y = X_tv.y
    groups = X_tv.formula

    for test_ix, val_ix in splitter.split(X, y, groups):
        X_test, X_val = X.iloc[test_ix, :].copy(deep=True), X.iloc[val_ix, :].copy(deep=True)

    return [X_train, X_test, X_val]

In [194]:
def confuse(obs_y, theo_y):
    # Copy from confuse ipynb
    con = confusion_matrix(list(obs_y), list(theo_y))
    if con.shape == (1, 1):
        print('error!')

    elif con.shape == (2, 2):
        tn, fp, fn, tp = con.ravel()
        sens = tpr = tp / (tp + fn)
        spec = tnr = tn / (tn + fp)
        f1 = (2 * tp) / (2 * tp + fp + fn)
        acc = (tp + tn) / (tp + tn + fp + fn)
        confuse = {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp}
        prec = tp / (tp + fp)
              
        return [acc, {'sens': sens, 'spec': spec, 'f1': f1,
                      'test_n': tn + fp + fn + tp, 'test_true': tp + fp,
                      'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn
                     }]

    else:
        print('error!')

In [195]:
def direct_model(model, dfs):
    train_df = dfs[0]
    test_df = dfs[1]
    
    acc_train = confuse(np.array(train_df.y), np.array(train_df.X))[0]
    result = confuse(np.array(test_df.y), np.array(test_df.X))
    acc_test = result[0]
    result_dict = result[1]
    result_dict['acc_train'] = acc_train
    result_dict['acc_test'] = acc_train
                     
    return result_dict

In [437]:
def ml_model(ml_model, dfs):
    train_df = dfs[0]  
    train_X = train_df.X.to_numpy()
    train_X = np.stack(train_X)
    train_y = np.array(train_df.y)
    train_w = np.array(train_df.w)

    test_df = dfs[1]
    test_X = test_df.X.to_numpy()
    test_X = np.stack(test_X)
    test_y = np.array(test_df.y)
    test_w = np.array(test_df.w)
    
    if ml_model is 'random_forest':
        # https://stackoverflow.com/questions/30805192/scikit-learn-random-forest-class-weight-and-sample-weight-parameters
        # https://scikit-learn.org/stable/auto_examples/svm/plot_weighted_samples.html
        clf = rfc(max_features=32, n_estimators=100, random_state=0, 
                  class_weight="balanced", n_jobs=-1) #"balanced"  {0:1,1:5}
        
    elif ml_model is 'XGBoost':
        # https://xgboost.readthedocs.io/en/latest/get_started.html
        # https://www.datacamp.com/community/tutorials/xgboost-in-python       
        clf = xgb.XGBClassifier('multi:softmax', eta=0.2, min_child_weight=1,
                                gamma=0, num_class=1, eval_metric='error')
    
    elif ml_model is 'ThunderSVM':
        # Copied from vanilla scikit SVM parameters
        clf = SVC(kernel='linear', C=10, gamma=1)
        
    model = clf.fit(train_X, train_y, sample_weight=train_w) 
    acc_train = model.score(train_X, train_y)
    acc_test = model.score(test_X, test_y)
    predict_y = model.predict(test_X)
    result_dict = confuse(test_y, predict_y)[1]
    result_dict['acc_train'] = acc_train
    result_dict['acc_test'] = acc_train
    
    
    '''
    scores = cross_val_score(model, theo_x,
                                 obs_y,
                                 cats,
                                 cv=GroupKFold(n_splits=5)
                                 )
    cross_a = float(scores.mean())
    cross_s = float(scores.std())
    '''
    
    return result_dict
    

In [197]:
def deep_chem_model(dfs):
    # How to call deep chem?
    pass

In [ ]:
# Make function eventually!

# Assumes only one target (e.g. water) at a time!
target_fdr = 'fdr_' + target[0]
target_coloc = 'colocalization_' + target[0]
join_df['best_fdr'] = join_df[['fdr', target_fdr]].min(axis=1)

dict_dict = {}
for index, row in filter_param_df.iterrows():
    print('start ' + str(index))
    filtered_df = join_df[(join_df.polarity == row.polarity) &
                         (join_df.best_fdr <= row.fdr) &
                         ((join_df[target_coloc] >= row.coloc) | (join_df[target_coloc] == 0))
                         ]
    
    xyw_df = filtered_df[['formula', 'hmdb_ids', 'ds_id', row.X, row.y, row.w]].copy(deep=True)    
    
    if row.X != row.y:
        xyw_df = xyw_df.rename(columns={row.X: 'X', row.y: 'y', row.w: 'w'}, inplace=False)
    
    else:
        # Control case with perfect prediction/memorization!
        xyw_df['temp_X'] = filtered_df[row.y]
        xyw_df['temp_y'] = filtered_df[row.y]
        xyw_df = xyw_df[['formula', 'hmdb_ids', 'ds_id', 'temp_X', 'temp_y', row.w]].copy(deep=True)
        xyw_df = xyw_df.rename(columns={'temp_X': 'X', 'temp_y': 'y', row.w: 'w'}, inplace=False)
        
    xyw_df = sanitize(xyw_df)
   
    split_dfs = split(xyw_df, single_fold_group)
    
    if row.mclass == 'direct':
        result = direct_model(row.model, split_dfs)
        all_line = {**dict(row), **result}
        dict_dict[index] = all_line
        
    if row.mclass == 'ml':
        result = ml_model(row.model, split_dfs)      
        all_line = {**dict(row), **result}
        dict_dict[index] = all_line
        
    if row.mclass == 'dc':
        result = dc_model(row.model, split_dfs)
        all_line = {**dict(row), **result}
        dict_dict[index] = all_line

results = pd.DataFrame.from_dict(dict_dict, orient='index')
print('Run complete!')

start 0


In [427]:
ml_random_forest = results

In [428]:
ml_random_forest.round(3)

,target,polarity,fdr,coloc,mclass,model,X,y,w,sens,spec,f1,test_n,test_true,tp,tn,fp,fn,acc_train,acc_test
0,H2O,positive,0.05,0.75,ml,random_forest,bits,n_loss_wparent_H2O,weight,0.485,0.629,0.197,140476,53618,6622,79834,46996,7024,0.768,0.768
1,H2O,positive,0.05,0.75,ml,random_forest,mord_norm,n_loss_wparent_H2O,weight,0.323,0.896,0.282,140476,17587,4403,113646,13184,9243,0.826,0.826
2,H2O,positive,0.05,0.75,ml,random_forest,fp_feats,n_loss_wparent_H2O,weight,0.335,0.869,0.262,140476,21230,4570,110170,16660,9076,0.826,0.826
